# Market Basket Analysis - Memory Efficient Version

This notebook performs market basket analysis on retail data using a memory-efficient approach.

In [26]:
import pandas as pd
import numpy as np
import requests
import os
from itertools import combinations
import collections

## 1. Data Loading

In [27]:
# Configuration
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx'
raw_data_path = '../data/01_raw/online_retail_II.xlsx'

# Create directory and download if needed
os.makedirs(os.path.dirname(raw_data_path), exist_ok=True)

if not os.path.exists(raw_data_path):
    print('Downloading dataset...')
    response = requests.get(url)
    response.raise_for_status()
    with open(raw_data_path, 'wb') as f:
        f.write(response.content)
    print('Download complete.')
else:
    print('Dataset already exists.')

Dataset already exists.


In [28]:
# Load data from both sheets
print('Loading data...')
xls = pd.ExcelFile(raw_data_path)
df_year_1 = pd.read_excel(xls, 'Year 2009-2010')
df_year_2 = pd.read_excel(xls, 'Year 2010-2011')

# Combine datasets
df = pd.concat([df_year_1, df_year_2], ignore_index=True)
print(f'Total data shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
df.head()

Loading data...
Total data shape: (1067371, 8)
Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 2. Data Preprocessing & Sampling

In [29]:
# Basic cleaning
print('Original shape:', df.shape)

# Remove rows with missing CustomerID
df = df.dropna(subset=['Customer ID'])
print('After removing null CustomerID:', df.shape)

# Remove returns (invoices starting with 'C')
df = df[~df['Invoice'].astype(str).str.startswith('C')]
print('After removing returns:', df.shape)

# Keep only positive quantities
df = df[df['Quantity'] > 0]
print('After removing negative quantities:', df.shape)

# Clean descriptions
df['Description'] = df['Description'].str.strip()
df = df.dropna(subset=['Description'])
print('After cleaning descriptions:', df.shape)

Original shape: (1067371, 8)
After removing null CustomerID: (824364, 8)
After removing returns: (805620, 8)
After removing negative quantities: (805620, 8)
After cleaning descriptions: (805620, 8)


In [30]:
# ULTRA-CONSERVATIVE SAMPLING for guaranteed success
print('\n=== SMART SAMPLING FOR MEMORY EFFICIENCY ===')

# Step 1: Take only top 50 most popular products
product_counts = df['Description'].value_counts()
top_products = product_counts.head(50).index.tolist()

print(f'Top 50 products:')
for i, (product, count) in enumerate(product_counts.head(50).items(), 1):
    print(f'{i:2d}. {product[:50]:<50} ({count:,} times)')

# Filter to only these products
df_sample = df[df['Description'].isin(top_products)].copy()
print(f'\nData with top 50 products: {df_sample.shape}')

# Step 2: Sample 10000 random invoices
unique_invoices = df_sample['Invoice'].unique()
sample_invoices = np.random.choice(unique_invoices, size=min(10000, len(unique_invoices)), replace=False)
df_final = df_sample[df_sample['Invoice'].isin(sample_invoices)].copy()

print(f'\nFinal sample: {df_final.shape}')
print(f'Unique invoices: {df_final["Invoice"].nunique()}')
print(f'Unique products: {df_final["Description"].nunique()}')



=== SMART SAMPLING FOR MEMORY EFFICIENCY ===
Top 50 products:
 1. WHITE HANGING HEART T-LIGHT HOLDER                 (5,181 times)
 2. REGENCY CAKESTAND 3 TIER                           (3,430 times)
 3. ASSORTED COLOUR BIRD ORNAMENT                      (2,777 times)
 4. JUMBO BAG RED RETROSPOT                            (2,702 times)
 5. REX CASH+CARRY JUMBO SHOPPER                       (2,141 times)
 6. PARTY BUNTING                                      (2,122 times)
 7. LUNCH BAG  BLACK SKULL.                            (2,117 times)
 8. LUNCH BAG SPACEBOY DESIGN                          (1,941 times)
 9. HOME BUILDING BLOCK WORD                           (1,929 times)
10. STRAWBERRY CERAMIC TRINKET BOX                     (1,922 times)
11. HEART OF WICKER SMALL                              (1,899 times)
12. 60 TEATIME FAIRY CAKE CASES                        (1,880 times)
13. HEART OF WICKER LARGE                              (1,864 times)
14. LUNCH BAG CARS BLUE                 

## 3. Market Basket Analysis

In [31]:
# Simple co-occurrence analysis (no memory issues)
print('\n=== MARKET BASKET ANALYSIS ===')

# Group products by invoice
invoice_products = df_final.groupby('Invoice')['Description'].apply(list).reset_index()
print(f'Analyzing {len(invoice_products)} invoices...')

# Count co-occurrences
pair_counts = collections.Counter()
product_counts = collections.Counter()
total_baskets = 0

for products in invoice_products['Description']:
    if len(products) >= 2:  # Only multi-item baskets
        total_baskets += 1
        
        # Count individual products
        for product in products:
            product_counts[product] += 1
        
        # Count pairs
        for pair in combinations(sorted(products), 2):
            pair_counts[pair] += 1

print(f'Multi-item baskets: {total_baskets}')
print(f'Unique product pairs: {len(pair_counts)}')


=== MARKET BASKET ANALYSIS ===
Analyzing 10000 invoices...
Multi-item baskets: 7118
Unique product pairs: 1274


In [32]:
# Calculate association metrics
associations = []

for (product_a, product_b), pair_count in pair_counts.items():
    # Support: P(A and B)
    support = pair_count / total_baskets
    
    # Confidence: P(B|A) and P(A|B)
    confidence_a_to_b = pair_count / product_counts[product_a]
    confidence_b_to_a = pair_count / product_counts[product_b]
    
    # Lift: Support / (P(A) * P(B))
    prob_a = product_counts[product_a] / total_baskets
    prob_b = product_counts[product_b] / total_baskets
    lift = support / (prob_a * prob_b)
    
    associations.append({
        'Product_A': product_a,
        'Product_B': product_b,
        'Count': pair_count,
        'Support': support,
        'Confidence_A→B': confidence_a_to_b,
        'Confidence_B→A': confidence_b_to_a,
        'Lift': lift
    })

# Create results DataFrame
results_df = pd.DataFrame(associations)
results_df = results_df.sort_values(['Lift', 'Support'], ascending=[False, False])

print(f'\nTotal associations found: {len(results_df)}')
print('\nTop 15 Product Associations:')
display(results_df.head(15).round(3))


Total associations found: 1274

Top 15 Product Associations:


,Product_A,Product_B,Count,Support,Confidence_A→B,Confidence_B→A,Lift
59,STRAWBERRY CERAMIC TRINKET BOX,SWEETHEART CERAMIC TRINKET BOX,396,0.056,0.583,0.839,8.795
1077,PAPER CHAIN KIT 50'S CHRISTMAS,PAPER CHAIN KIT VINTAGE CHRISTMAS,346,0.049,0.566,0.716,8.345
122,CHOCOLATE HOT WATER BOTTLE,HOT WATER BOTTLE TEA AND SYMPATHY,310,0.044,0.534,0.598,7.344
165,WOODEN FRAME ANTIQUE WHITE,WOODEN PICTURE FRAME WHITE FINISH,411,0.058,0.623,0.675,7.278
567,COOK WITH WINE METAL SIGN,GIN + TONIC DIET METAL SIGN,222,0.031,0.494,0.456,7.227
324,COOK WITH WINE METAL SIGN,HAND OVER THE CHOCOLATE SIGN,208,0.029,0.463,0.416,6.595
3,HOME BUILDING BLOCK WORD,LOVE BUILDING BLOCK WORD,374,0.053,0.544,0.635,6.569
572,COOK WITH WINE METAL SIGN,PLEASE ONE PERSON METAL SIGN,209,0.029,0.465,0.413,6.548
857,HEART OF WICKER LARGE,HEART OF WICKER SMALL,413,0.058,0.612,0.581,6.125
232,GIN + TONIC DIET METAL SIGN,HAND OVER THE CHOCOLATE SIGN,209,0.029,0.429,0.418,6.109


## 4. Business Insights

In [33]:
# Filter for strong associations
strong_associations = results_df[
    (results_df['Support'] >= 0.01) &  # At least 1% support
    (results_df['Lift'] >= 1.5)        # At least 50% lift
]

print('\n' + '='*60)
print('BUSINESS INSIGHTS')
print('='*60)

if len(strong_associations) > 0:
    print(f'\n🎯 STRONG ASSOCIATIONS FOUND: {len(strong_associations)}')
    print('\nTop recommendations for cross-selling:')
    
    for i, row in strong_associations.head(10).iterrows():
        print(f'\n{len(strong_associations.head(10)) - len(strong_associations.head(10)[strong_associations.head(10).index <= i]) + 1}. "{row["Product_A"]}" + "{row["Product_B"]}"')
        print(f'   • Lift: {row["Lift"]:.2f}x more likely to be bought together')
        print(f'   • Confidence: {max(row["Confidence_A→B"], row["Confidence_B→A"]):.1%} of customers who buy one also buy the other')
        print(f'   • Support: {row["Support"]:.1%} of all transactions contain both items')
    
    print(f'\n📊 SUMMARY STATISTICS:')
    print(f'• Average lift for strong associations: {strong_associations["Lift"].mean():.2f}')
    print(f'• Highest lift found: {strong_associations["Lift"].max():.2f}')
    print(f'• Most frequent pair appears in {strong_associations["Support"].max():.1%} of transactions')
    
else:
    print('\n⚠️  No strong associations found with current thresholds.')
    print('\nTop 5 associations by lift (regardless of thresholds):')
    
    for i, row in results_df.head(5).iterrows():
        print(f'\n{i+1}. "{row["Product_A"]}" + "{row["Product_B"]}"')
        print(f'   • Lift: {row["Lift"]:.2f}')
        print(f'   • Support: {row["Support"]:.1%}')

print(f'\n💡 RECOMMENDATIONS:')
print('• Use these associations for product placement and bundling')
print('• Consider promotional campaigns for high-lift pairs')
print('• Monitor inventory levels for associated products')
print('• Implement "customers who bought X also bought Y" recommendations')


BUSINESS INSIGHTS

🎯 STRONG ASSOCIATIONS FOUND: 152

Top recommendations for cross-selling:

9. "STRAWBERRY CERAMIC TRINKET BOX" + "SWEETHEART CERAMIC TRINKET BOX"
   • Lift: 8.80x more likely to be bought together
   • Confidence: 83.9% of customers who buy one also buy the other
   • Support: 5.6% of all transactions contain both items

1. "PAPER CHAIN KIT 50'S CHRISTMAS" + "PAPER CHAIN KIT VINTAGE CHRISTMAS"
   • Lift: 8.35x more likely to be bought together
   • Confidence: 71.6% of customers who buy one also buy the other
   • Support: 4.9% of all transactions contain both items

8. "CHOCOLATE HOT WATER BOTTLE" + "HOT WATER BOTTLE TEA AND SYMPATHY"
   • Lift: 7.34x more likely to be bought together
   • Confidence: 59.8% of customers who buy one also buy the other
   • Support: 4.4% of all transactions contain both items

7. "WOODEN FRAME ANTIQUE WHITE" + "WOODEN PICTURE FRAME WHITE FINISH"
   • Lift: 7.28x more likely to be bought together
   • Confidence: 67.5% of customers who

In [34]:
# Save results
output_path = '../data/02_processed/market_basket_results.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
results_df.to_csv(output_path, index=False)
print(f'\nResults saved to: {output_path}')
print(f'Total associations: {len(results_df)}')
print('\nAnalysis complete! ✅')


Results saved to: ../data/02_processed/market_basket_results.csv
Total associations: 1274

Analysis complete! ✅
